In [16]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from imblearn.combine import SMOTEENN
from collections import Counter
import math
from sklearn.dummy import DummyClassifier

pd.options.mode.chained_assignment = 'warn'

UNREST_COLUMNS = ["EVENT_ID_CNTY", 
                "EVENT_DATE", 
                "EVENT_TYPE", 
                "REGION", 

                "TIMESTAMP"]

CASES_COLUMNS = ["iso_code",
                "continent",
                "location", 
                "date", 
                "total_cases", 
                "new_cases", 
                "total_deaths", 
                "reproduction_rate", 
                "hosp_patients", 
                "positive_rate", 
                "stringency_index", 
                "population",
                "median_age",
                "gdp_per_capita",
                "life_expectancy",
            ]
def multiPrecision(y, y_pred, **kwargs):
    return precision_score(y, y_pred, average="micro", **kwargs)

def multiRecall(y, y_pred, **kwargs):
    return precision_score(y, y_pred, average="micro", **kwargs)

def serialize(dataFrame, column):
    return [x for x in range(len(dataFrame[column].unique()))]

#Replace a column in a dataframe with a serialized version
def replaceDict(dataFrame, column):
    vals = serialize(dataFrame, column)
    return dict(zip(dataFrame[column].unique(), vals))

def oneToOne(df, column):
    count = []
    ret = pd.DataFrame()
    
    count = [{"col": val, "count": df[df[column] == val].shape[0]} for val in df[column].unique()]
    count.sort(key=lambda x: x.get("count"))
    i = 1
    for col in count:
        if ret.empty:
            ret = pd.DataFrame(df[df[column] == col.get("col")].sample(math.floor(count[0].get("count") * i), replace=True).drop_duplicates())
        else:
            ret = ret.append(df[df[column] == col.get("col")].sample(math.floor(count[0].get("count") * i), replace=True).drop_duplicates())
        i = i + 10
    return ret

def manyToOne(df, column):
    count = []
    ret = pd.DataFrame()
    
    count = [df[df[column] == val].shape[0] for val in df[column].unique()]
    count.sort()

def randomSample(one=True):
    #Read in data from files
    unrest_df = pd.read_csv("./coronavirus_Oct31.csv")
    #Remove rows with event type of strategic developments
    unrest_df = unrest_df[unrest_df.EVENT_TYPE != 'Strategic developments']
 
    covid_cases_df = pd.read_csv("./owid-covid-data.csv")

    unrest_df = unrest_df[unrest_df.columns.intersection(UNREST_COLUMNS)]
    covid_cases_df = covid_cases_df[covid_cases_df.columns.intersection(CASES_COLUMNS)]
    #Get data based on the input iso country codes
    #unrest = multiContains(unrest_df, "EVENT_ID_CNTY", isoCodes)
    #cases = multiSearch(covid_cases_df, 'iso_code', isoCodes)

    #Convert "date" type columns to dates
    unrest_df.EVENT_DATE = pd.to_datetime(unrest_df.EVENT_DATE)
    unrest_df.EVENT_ID_CNTY = unrest_df.EVENT_ID_CNTY.astype(str).str[:3]
    covid_cases_df.date = pd.to_datetime(covid_cases_df.date)
    #Merge the two datasets with an inner join on the date fields
    merge = unrest_df.merge(covid_cases_df, how="inner", left_on=["EVENT_DATE", 'EVENT_ID_CNTY'], right_on=["date", "iso_code"])
    #merge = merge.drop_duplicates(subset=['EVENT_DATE', 'EVENT_ID_CNTY'])
    #Drop the iso code to avoid duplicates
    
    merge = merge.drop(['EVENT_ID_CNTY'], axis=1)
    #Drop remaining duplicates
    #merge = merge.drop_duplicates()
    print(merge.shape)
    #Get the list of event types in this particular set of data
    #issueType = merge['EVENT_TYPE']
    #Serialize the data and return it as the expected values for training
    #issueType = issueType.replace(replaceDict(unrest_df, "EVENT_TYPE"))
    result = pd.DataFrame()
    if one:
        result = oneToOne(merge, 'EVENT_TYPE')
    else:
        result = merge
    issueType = result['EVENT_TYPE']
    #Serialize the data and return it as the expected values for training
    issueType = issueType.replace(replaceDict(unrest_df, "EVENT_TYPE"))

    #Drop remaining unneeded data
    result = result.drop(['EVENT_TYPE', 'EVENT_DATE', 'REGION', 'iso_code', 'continent', 'location', 'date', 'TIMESTAMP'], axis=1).fillna(0)

    return result, issueType

min_max_scaler = preprocessing.MinMaxScaler()

X, y = randomSample(False)
X = min_max_scaler.fit_transform(X)

X_train, y_train, X_test, y_test = train_test_split(X, y, test_size=0.2)

resampler = SMOTEENN()
X_train, y_train = resampler.fit_resample(X_train, y_train)

model = MLPClassifier(hidden_layer_sizes=5, alpha=1.0/5).fit(X_train, y_train)
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

print("training\n", classification_report(issueType, y_train_pred))
print(confusion_matrix(y_train, y_train_pred))

print("test\n", classification_report(testIssues, y_test_pred))
print(confusion_matrix(y_test, y_test_pred))


(14596, 19)


ValueError: Unknown label type: 'continuous-multioutput'

In [3]:
import matplotlib.pyplot as plt

# plt.rc()